# Provision Knowledge source and knowledge base 
This notebook configures required knowledge sources and knowledge base for the Fault Diagnosis Agent

## Overview
- **Knowledge Source**: specifies the content used for agentic retrieval.It either encapsulates a search index which is populated by an external data source, or it's a direct connection to a remote source such as Bing or Sharepoint that's queried directly 
- **Knowledge Base**: a top-level object that orchestrates agentic retrieval. It defines which knowledge sources to query and the default behavior for retrieval operations
- **Project Connection**: a **RemoteTool** connection on your Microsoft Foundry project allowing the agent to securely communicate with Azure AI Search for retrieval operations.

## 1. Import dependencies and load environment
Import all neccessary libraries for the provisioning

In [ ]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import AzureBlobKnowledgeSource, AzureBlobKnowledgeSourceParameters, KnowledgeBaseAzureOpenAIModel, AzureOpenAIVectorizerParameters, KnowledgeSourceAzureOpenAIVectorizer, KnowledgeSourceContentExtractionMode, KnowledgeSourceIngestionParameters
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Configuration
storage_connection_string = os.environ.get("AZURE_STORAGE_CONNECTION_STRING")
search_endpoint = os.environ.get("SEARCH_SERVICE_ENDPOINT")
search_key = os.environ.get("SEARCH_ADMIN_KEY")
model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")
openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
openai_key = os.environ.get("AZURE_OPENAI_KEY")
knowledge_source_name = "machine-wiki-blob-ks"



## 2. Create Knowledge Source
Creates a Blob Storage knowledge source

In [ ]:

index_client = SearchIndexClient(
    endpoint=search_endpoint, credential=AzureKeyCredential(search_key))

knowledge_source = AzureBlobKnowledgeSource(
    name=knowledge_source_name,
    description="This knowledge source pulls from a blob storage container.",
    encryption_key=None,
    azure_blob_parameters=AzureBlobKnowledgeSourceParameters(
        connection_string=storage_connection_string,
        container_name="machine-wiki",
        folder_path=None,
        is_adls_gen2=False,
        ingestion_parameters=KnowledgeSourceIngestionParameters(
            identity=None,
            disable_image_verbalization=False,
            chat_completion_model=KnowledgeBaseAzureOpenAIModel(
                azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                    resource_url=openai_endpoint,
                    deployment_name=model_deployment_name,
                    api_key=openai_key,
                    model_name=model_deployment_name
                )
            ),
            embedding_model=KnowledgeSourceAzureOpenAIVectorizer(
                azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                    resource_url=openai_endpoint,
                    deployment_name="text-embedding-ada-002",
                    api_key=openai_key,
                    model_name="text-embedding-ada-002"
                )
            ),
            content_extraction_mode=KnowledgeSourceContentExtractionMode.MINIMAL,
            ingestion_schedule=None,
            ingestion_permission_options=None
        )
    )
)

index_client.create_or_update_knowledge_source(knowledge_source)
print(
    f"Knowledge source '{knowledge_source.name}' created or updated successfully.")

## 3. Examine the Knowledge Source
The knowledge source we created is powered by **Azure Search**. Navigate to [Azure Portal] 